In [2]:
!pip install -q crewai


In [3]:
!pip install -q 'crewai[tools]'

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.3/463.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 8.1 MB/s eta 0:00:00


In [4]:
!pip install python-dotenv

In [40]:
!pip uninstall langchain
!pip uninstall langchain-groq

Found existing installation: langchain 0.2.16
Uninstalling langchain-0.2.16:
  Would remove:
    /usr/local/bin/langchain-server
    /usr/local/lib/python3.10/dist-packages/langchain-0.2.16.dist-info/*
    /usr/local/lib/python3.10/dist-packages/langchain/*
Proceed (Y/n)? y
  Successfully uninstalled langchain-0.2.16
Found existing installation: langchain-groq 0.2.0
Uninstalling langchain-groq-0.2.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/langchain_groq-0.2.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/langchain_groq/*
Proceed (Y/n)? Y
  Successfully uninstalled langchain-groq-0.2.0


In [41]:
!pip install langchain
!pip install langchain-groq==0.1.9

  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_text_splitters-0.3.0-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.4
    Uninstalling langchain-text-splitters-0.2.4:
      Successfully uninstalled langchain-text-splitters-0.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.17 requires langchain<0.3.0,>=0.2.16, but you have langchain 0.3.0 which is incompatible.
langchain-community 0.2.17 requires langchain-core<0.3.0,>=0.2.39, but you have langchain-core 0.3.0 which is incompatible.
langchain-experimental 0.0.65 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.0 which is incompatible.
  Using cach

In [38]:
from google.colab import userdata
open_api_key = userdata.get('OPENAI_API_KEY')
serper_api_key = userdata.get('SERPER_API_KEY')
grok_api_key = userdata.get('GROK_API_KEY')
print(open_api_key)
print(serper_api_key)

sk-proj-kDeCpuNA7kaWDMdFYSqAT3BlbkFJ5K3FdXYVndCRNBOwC3u4
63c0a7b5eb3fca1946d9e8730d55dba9d05ad868


In [9]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

In [10]:
search_tool = SerperDevTool()

In [42]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

llm = ChatGroq(model="mixtral-8x7b-32768",groq_api_key=grok_api_key,temperature=0,verbose=True)

In [44]:
#Define agent with roles and goals
researcher = Agent(
    role = "Senior Researcher",
    goal = "Look up latest advancements in AI Agents",
    backstory = """
    You are working at a leading AI company.
    Your expertise lies in searching Google and finding latest advancements in AI Agent frameworks.
    You dissect complex data and present actionable insights.
    """,
    verbose = True,
    allow_delegation = False,
    tools = [search_tool],
    llm=ChatGroq(model="mixtral-8x7b-32768",groq_api_key=grok_api_key,temperature=0.3,verbose=True)
)

In [45]:
writer = Agent(
    role = "Tech Content Strategist",
    goal = "Summarise latest advancements in AI agents in a concise article",
    backstory = """
    You are a creative content strategist, known for your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    """,
    verbose = True,
    allow_delegation = True,
    llm=ChatGroq(model="mixtral-8x7b-32768",groq_api_key=grok_api_key,temperature=0.7,verbose=True)
)

In [48]:
#Create tasks for your agents
task1 = Task(
    description = """
    Conduct a comprehensive anaylsis of latest advancements in Ai Agents in recent months.
    Identify key trends, major breakthroughs and potential challenges and industry impact.
    """,
    expected_output = "Full analysis report in bullet points",
    agent = researcher
)

In [49]:
task2 = Task(
    description = """
    Using the insights provided, write a short article that highlights the most significant AI agent advancements.
    Your post should be informative yet accessible, caterign to a tech-savvy audience.
    Avoid complex terms for that it doesn't sound like AI.
    """,
    expected_output = "Full blog post of atleast 3 paragraphs",
    agent = writer
)

In [50]:
crew = Crew(
    agents = [researcher, writer],
    tasks = [task1, task2],
    verbose = 1
)


In [51]:
result = crew.kickoff()
print(result)

 [2024-09-16 11:14:11][DEBUG]: == Working Agent: Senior Researcher
 [2024-09-16 11:14:11][INFO]: == Starting Task: 
    Conduct a comprehensive anaylsis of latest advancements in Ai Agents in recent months.
    Identify key trends, major breakthroughs and potential challenges and industry impact.
    


> Entering new CrewAgentExecutor chain...
To conduct a comprehensive analysis of the latest advancements in AI agents, I need to search the internet for recent articles, research papers, and news related to AI agent frameworks. I will then examine the key trends, major breakthroughs, potential challenges, and industry impact.

Action: Search the internet
Action Input: {'search_query': 'latest advancements in AI agent frameworks from the last 6 months'}
 


Search results: Title: A Quick Review of The Most Popular AI Agent Frameworks (June ...
Link: https://medium.com/@ceo_44783/a-quick-review-of-the-most-popular-ai-agent-frameworks-june-2024-ce53c0ef809a
Snippet: 1. Autogen · 2. Semanti